In [ ]:
%load_ext autoreload
%autoreload
%matplotlib widget
import importlib
from SOAEpeaks import load_df
import phaseco as pc
from phaseco import *
from tqdm import tqdm
import numpy as np
import pickle
import matplotlib.pyplot as plt
import scipy as sp
from scipy.signal.windows import gaussian
importlib.reload(pc)


In [ ]:


# Get different species
df = load_df(laptop=True, dfs_to_load=["Curated Data"])
wf_list = []
for species in ['Anolis', 'Owl', 'Human']:
    df_species = df[df['species'] == species]
    for i in range (3):
        row = df_species.iloc[i]
        wf_fn = row['filepath'].split('\\')[-1]
        wf_list.append((row['wf'], row['sr'], wf_fn, species))
del df

In [ ]:
wf_idx = 7 # Started with 0, 2 (Lizard) - 4, 5 (Owl) - 6, 7 (Human)

wf, fs, wf_fn, species = wf_list[wf_idx]

# Set parameters (same for human and lizard)
tau = 2**12 / 44100 # Everyone uses the same tau
tauS = int(tau*fs)
# Raise warning if tauS is not a power of two AND the samplerate is indeed 44100
if np.log2(tauS) != int(np.log2(tauS)):
    if fs == 44100:
        raise ValueError("tauS is not a power of 2, but the samplerate is 44100!")
    else:
        print(f"WARNING: tauS is not a power of 2, but the samplerate is {fs} (not 44100), so we'll assume you're just ensuring tau aligns with other waveforms!")

rho = 0.7

if species == 'Human':
    # Human parameters
    min_xi = 0.001
    max_xi = 1
    delta_xi = 0.001
    if wf_fn == 'human_TH14RearwaveformSOAE.mat':
        max_xi = 2.5
    max_khz = 6
elif species in ['Lizard', 'Anolis']:
    # Lizard parameters
    min_xi = 0.001
    max_xi = 0.1
    delta_xi = 0.0005
    max_khz = 6
elif species == 'Owl':
    # Owl parameters
    min_xi = 0.001
    max_xi = 0.1
    delta_xi = 0.0005
    max_khz = 12

In [ ]:


# Calculate colossogram (if needed)
if 0:
    operation = 'generate+save'
else:
    operation = 'open'

if operation == 'generate+save':
    num_xis = int((max_xi - min_xi) / delta_xi) + 1
    xis = np.linspace(min_xi, max_xi, num_xis)

    max_xiS = max(xis) * fs
    f = rfftfreq(tauS, 1/fs)
    # Make sure we have a consistent number of segments to take vector strenght over since this will change with xi
    N_segs = int((len(wf) - tauS) / max_xiS)    

    coherences = np.zeros((len(f), len(xis)))
    print(f"Calculating coherence for wf {wf_fn} with {len(xis)} xi values")
    for i, xi in enumerate(tqdm(xis)):
        coherences[:, i] = get_coherence(wf=wf, fs=fs, tauS=tauS, xi=xi, ref_type="next_seg", N_segs=N_segs, rho=rho)[1]

In [ ]:
# Save/open coherences
suptitle=rf"[{wf_fn}]   [$\rho$={rho}]   [$\tau$={tau*1000:.2f}ms]" 
fn_id = rf"tau={tau*1000:.0f}, rho={rho}, {species}, {wf_fn.split('.')[0]}"
pkl_fn = f'C_xi Decay Coherences - {fn_id}'
pkl_folder = r'Pickles/'
fig_folder = r'Colossogram Figures/C_xi Decay Figures/'

if operation == 'generate+save':
    with open(pkl_folder + pkl_fn + '.pkl', 'wb') as file:
        pickle.dump((coherences, f, xis, tau, rho, wf_fn, species), file)
elif operation == 'open':
    with open(pkl_folder + pkl_fn + '.pkl', 'rb') as file:
        coherences, f, xis, tau, rho, wf_fn, species = pickle.load(file)

In [ ]:
# Peak pick the target bins
if wf_fn == 'anole_AC6rearSOAEwfB1.mat': # 0
    peak_freqs = [1225, 2150, 4300]
    noise_freqs = [400, 12000]
elif wf_fn == 'anole_ACsb18learSOAEwfG4.mat': # 2
    peak_freqs = [990, 2000, 3670]
    noise_freqs = [400, 12000]
elif wf_fn == 'owl_Owl6L1.mat': # 4
    peak_freqs = [4867, 6384, 7235]
    noise_freqs = [400, 12000]
elif wf_fn == 'owl_TAG4learSOAEwf1.mat': # 5
    peak_freqs = [6280, 7820, 10487]
    noise_freqs = [400, 12000]
elif wf_fn == 'human_TH14RearwaveformSOAE.mat': # 6
    peak_freqs = [603, 2250, 4370]
    noise_freqs = [400, 12000]
elif wf_fn == 'human_TH21RearwaveformSOAE.mat': # 7
    peak_freqs = [2000, 2605, 4135]
    noise_freqs = [400, 12000]
else:
    raise(Exception("Haven't peak picked this waveform yet!"))
    

bin_idxs = []
bin_names = []

for peak_freq in peak_freqs:
    bin_idxs.append(np.argmin(np.abs(f - peak_freq)))
    bin_names.append(f"{peak_freq:.0f}Hz Peak")

for noise_freq in noise_freqs:
    bin_idxs.append(np.argmin(np.abs(f - noise_freq)))
    bin_names.append(f"{noise_freq:.0f}Hz Noise")
    
freq_list = peak_freqs + noise_freqs

In [ ]:
# Plot colossogram
plt.close('all')
plt.figure(figsize=(15, 5))
plot_colossogram(coherences, f, xis, tau, max_khz=max_khz, cmap='magma')
plt.title(f"{species} Colossogram", fontsize=18)
plt.suptitle(suptitle, fontsize=10)
plt.savefig(f'{fig_folder}C_xi Colossogram - {fn_id}.png', dpi=300)
plt.show()

In [ ]:
# Plot a slice of the colossogram with target bins marked
target_xi = 0.02
xi_idx = np.argmin(np.abs(xis - target_xi))
coherence_slice = coherences[:, xi_idx]
psd = get_welch(wf=wf, fs=fs, tauS=tauS)[1]
# psd_xi = get_welch(wf=wf, fs=fs, tauS=tauS, xi=target_xi)[1]


plt.close('all')
plt.figure(figsize=(12, 5))
plt.suptitle(suptitle)
plt.subplot(2, 1, 1)
plt.title(rf"{species} Colossogram Slice at $\xi={xis[xi_idx]:.3f}$")
plt.plot(f / 1000, coherence_slice)
for freq, idx in zip(freq_list, bin_idxs):
    plt.scatter(f[idx] / 1000, coherence_slice[idx], c='r')
plt.xlabel("Frequency (kHz)")
plt.ylabel(r'$C_{\xi}$')
plt.xlim(0, max_khz)
plt.subplot(2, 1, 2)
plt.title(rf"{species} PSD")
plt.plot(f / 1000, np.log10(psd), label='PSD')
#plt.plot(f / 1000, np.log10(psd_xi), label=r'PSD ($\xi$)')
for freq, idx in zip(freq_list, bin_idxs):
    plt.scatter(f[idx] / 1000, np.log10(psd[idx]), c='r')
    #plt.scatter(f[idx] / 1000, np.log10(psd_xi[idx]), c='r')
plt.xlabel("Frequency (kHz)")
plt.ylabel("PSD [dB]")  
plt.legend()
plt.xlim(0, max_khz)
plt.tight_layout()
plt.savefig(f'{fig_folder}C_xi Decay Target Bins - {fn_id}.png', dpi=300)
plt.show()

In [ ]:
def get_is_signal(coherences, f, xis, f_target_idx, f_noise=12000, sample_hw=5, z_alpha=0.05, crop=False):
    f_noise_idx = (np.abs(f - f_noise)).argmin()  # find frequency bin index closest to our 12kHz cutoff
    # Get mean and std dev of coherence (over frequency axis, axis=0) for each xi value (using only "noise" frequencies above our cutoff)
    noise_means = np.mean(coherences[f_noise_idx:, :], axis=0) 
    noise_stds = np.std(coherences[f_noise_idx:, :], axis=0, ddof=1) # ddof=1 since we're using sample mean (not true mean) in sample std estimate
    # Now for each xi value, run a z-test to see if it's noise or not
    is_signal = np.full(len(xis), True, dtype=bool)
    for xi_idx in range(len(xis)):
        # Skip xi values that are too close to the edges to get a full sample
        if xi_idx < sample_hw:
            is_signal[xi_idx] = True
            continue
        elif xi_idx >= len(xis) - sample_hw:
            is_signal[xi_idx] = False
        coherence_sample = coherences[f_target_idx, xi_idx - sample_hw: xi_idx + sample_hw]
        # Calculate z test statistic
        sample_mean = np.mean(coherence_sample)
        z = (sample_mean - noise_means[xi_idx]) / (noise_stds[xi_idx]/np.sqrt(len(coherence_sample)))
        # Calculate p-value for a one-tailed test (sf = survival function = 1 - cdf)
        p = sp.stats.norm.sf(z)
        is_signal[xi_idx] = p < z_alpha
    target_coherences=coherences[f_target_idx, :]
    if crop:
        is_signal = is_signal[sample_hw:-sample_hw]
        xis = xis[sample_hw:-sample_hw]
        target_coherences = target_coherences[f_target_idx, sample_hw:-sample_hw]
    return is_signal, xis, target_coherences

In [ ]:
# Compare xi scale vs # cycles scale

# Parameters
sample_hw = 10
z_alpha = 0.05 # Minimum p-value for z-test; we assume noise unless p < z_alpha (so higher z_alpha means more signal bins)

# Plotting parameters
fig_subfolder = r'/Xi vs # Cycles (Z-Test)/'
s_signal=1
s_noise=1
s_decayed = 100
marker_signal='o'
marker_noise='o'
marker_decayed='*'
edgecolor_signal=None
edgecolor_noise=None
edgecolor_decayed='black'
crop=False
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

plt.figure(figsize=(12, 10))
plt.suptitle(suptitle)

# Xi scale
plt.subplot(2, 1, 1)
for f_target_idx, bin_name, color in zip(bin_idxs, bin_names, colors):   
    alpha=0.3 if 'Noise' in bin_name else 1    
    is_signal, xis_plot, target_coherences_plot = get_is_signal(coherences, f, xis, f_target_idx, f_noise=12000, sample_hw=sample_hw, z_alpha=z_alpha, crop=crop)
    is_noise = ~is_signal
    plt.scatter(xis_plot[is_signal]*1000, target_coherences_plot[is_signal], s=s_signal, edgecolors=edgecolor_signal, marker=marker_signal, color=color, alpha=alpha)
    plt.scatter(xis_plot[is_noise]*1000, target_coherences_plot[is_noise], s=s_noise, color=color, label=bin_name, edgecolors=edgecolor_noise, alpha=alpha)
    # Find decayed point
    for i in range(len(is_noise)):
        if not crop and i < sample_hw: # These are automatically set to signal anyway but whate the heck
            continue
        if is_noise[i] == 1:
            decayed_idx = i
            break
    xi_decayed = xis_plot[decayed_idx]
    plt.scatter(xi_decayed*1000, target_coherences_plot[decayed_idx], s=s_decayed, marker=marker_decayed, color=color, edgecolors=edgecolor_decayed)
title = r"$C_{\xi}$ Decays"
plt.ylabel(r'$C_{\xi}$')           
plt.title(title)
plt.xlabel(r'$\xi$ [ms]')
plt.legend()

# # Num Cycles scale
# plt.subplot(2, 1, 2)
# title = r"$C_{\xi}$ Decays (# Cycles)"
# for f_target_idx, bin_name, color in zip(bin_idxs, bin_names, colors):
#     target_coherences = coherences[f_target_idx, :]
#     xi_decayed_idx, xi_decayed = xi_decayed(coherences, f, xis, f_target_idx)
#     nums_cycles = xis * f[f_target_idx] # Nondimensionalize
#     plt.ylabel(r'$C_{\xi}$')           
#     plt.title(title)
#     alpha=1
#     if 'Noise' in bin_name:
#         alpha=0.3
#     plt.scatter(nums_cycles, target_coherences, label=bin_name, alpha=alpha, s=s_coherences, color=color)
#     plt.scatter(nums_cycles[xi_decayed_idx], target_coherences[xi_decayed_idx], marker='*', s=s_decay, color=color, edgecolors='black', zorder=10)
# plt.xlabel(r'# of Cycles')
# # Prevent us from getting hella noise for the 12kHz one
# max_target_f_idx = np.max(bin_idxs[0:-2]) # Get the largest peak frequency
# max_num_cycles = xis[-1] * f[max_target_f_idx] # Get corresponding max number of cycles
# plt.xlim(0, max_num_cycles) 
# plt.legend()

# Book it!
plt.tight_layout()
plt.savefig(f'{fig_folder}{fig_subfolder}C_xi Decays Xi vs # Cycles (Z-Test) - {fn_id}.png', dpi=300)
plt.show()

In [ ]:
# # CROPPED Z-test 
# # Compare xi scale vs # cycles scale

# # Parameters
# sample_hw = 7
# z_alpha = 0.0000001

# # Plotting parameters
# fig_subfolder = r'/Xi vs # Cycles (Z-Test)/'
# s_signal=10
# s_noise=1
# marker_signal='*'
# marker_noise='o'
# edgecolor_signal=None
# edgecolor_noise=None
# colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# plt.figure(figsize=(12, 10))
# plt.suptitle(suptitle)

# # Xi scale
# plt.subplot(2, 1, 1)
# title = r"$C_{\xi}$ Decays"
# for f_target_idx, bin_name, color in zip(bin_idxs, bin_names, colors):   
#     # if 'Noise' not in bin_name:
#     #     continue
#     plt.ylabel(r'$C_{\xi}$')           
#     plt.title(title)
#     alpha=0.3 if 'Noise' in bin_name else 1    
#     xis_cropped = xis[sample_hw:len(xis)-sample_hw]
#     is_signal, xis_cropped, target_coherences_cropped = get_is_signal(coherences, f, xis, f_target_idx, f_noise=12000, sample_hw=sample_hw, z_alpha=z_alpha)
#     is_noise = ~is_signal
#     plt.scatter(xis_cropped[is_signal]*1000, target_coherences_cropped[is_signal], s=s_signal, edgecolors=edgecolor_signal, marker=marker_signal, color=color, alpha=alpha)
#     plt.scatter(xis_cropped[is_noise]*1000, target_coherences_cropped[is_noise], s=s_noise, color=color, label=bin_name, edgecolors=edgecolor_noise, alpha=alpha)
# plt.xlabel(r'$\xi$ [ms]')
# plt.legend()

In [ ]:
# DIPS
def xi_decayed(coherences, f, xis, f_target_idx, f_noise=12000, N_persistence=10, dip_prop=0.7):
    """
    Parameters
    ----------
    coherences : 2D array
        Coherence values for each frequency bin and xi value
    f : 1D array
        Frequency bins
    xis : 1D array
        xi values
    f_target_idx : int
        Index of target frequency
    f_noise : float, optional
        Noise cutoff frequency, by default 12kHz

    Returns
    -------
    xi_decayed : double
        xi value at which the peak decay is reached
    """
    f_noise_idx = (np.abs(f - f_noise)).argmin()  # find frequency bin index closest to our 12kHz cutoff
    coherences_noise = coherences[f_noise_idx:, :]
    noise_means = np.mean(coherences_noise, axis=0)
    noise_stds = np.std(coherences_noise, axis=0) # get standard deviation of coherence (over frequency axis, axis=0) for each xi value (using only "noise" frequencies above our cutoff)
    noise_floor = noise_means + noise_stds
    dips = coherences[f_target_idx, :] < noise_floor
    for xi_idx in range(len(xis)):
        if dips[xi_idx]:
            next_num_dips = sum(dips[xi_idx:xi_idx + N_persistence])
            if next_num_dips >= dip_prop * N_persistence:
                xi_decayed = xis[xi_idx]
                xi_decayed_idx = xi_idx
                return xi_decayed_idx, xi_decayed
    else:
        xi_decayed = xis[-1]
        xi_decayed_idx = len(xis)
        return xi_decayed_idx, xi_decayed


# Plot decays on various scales

s_coherences=1
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

for xscale_str in ['Xi', '# Cycles']:
    plt.figure(figsize=(12, 10))
    plt.suptitle(suptitle)
    for scale, idx in zip(['normal', 'log', 'gauss'], [1, 2, 3]):
        plt.subplot(3, 1, idx)
        for f_target_idx, bin_name, color in zip(bin_idxs, bin_names, colors):
            target_coherences = coherences[f_target_idx, :]
            xi_decayed_idx, xi_decayed = xi_decayed(coherences, f, xis, f_target_idx)
            title = r"$C_{\xi}$ Decays"
            if scale == 'normal':
                plt.ylabel(r'$C_{\xi}$')
            if scale == 'gauss':
                target_coherences = -np.sqrt(-np.log(target_coherences))
                plt.ylabel(r'$-\sqrt{-\ln(C_{\xi})}$')
                title+= " (Inverse Gaussian Scale)"
            elif scale == 'log':
                target_coherences = np.log(target_coherences)
                plt.ylabel(r'$\ln(C_{\xi})$')
                plt.title("Log Scale")
                title+= " (Log Scale)"            
            plt.title(title)
            z_alpha=1
            if 'Noise' in bin_name:
                z_alpha=0.3
            if xscale_str == '# Cycles':
                x = xis * f[f_target_idx] # Nondimensionalize
                xlabel = '# of Cycles'
            else:
                x = xis*1000
                xlabel = r'$\xi$ (ms)'
            plt.scatter(x, target_coherences, label=bin_name, alpha=z_alpha, s=s_coherences, color=color)
            plt.scatter(x[xi_decayed_idx], target_coherences[xi_decayed_idx], marker='x', label=f'{bin_name} Decay Xi', s=10, color=color)
        plt.xlabel(xlabel)
        plt.legend(loc='upper right')
    plt.tight_layout()
    plt.savefig(f'{fig_folder}C_xi Decay Scales - {xscale_str} - {fn_id}.png', dpi=300)
    plt.show()
    
    
# Compare xi scale vs # cycles scale

s_coherences=1
s_decay=100
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

plt.figure(figsize=(12, 10))
plt.suptitle(suptitle)

# Xi scale
plt.subplot(2, 1, 1)
title = r"$C_{\xi}$ Decays"
for f_target_idx, bin_name, color in zip(bin_idxs, bin_names, colors):
    target_coherences = coherences[f_target_idx, :]
    xi_decayed_idx, xi_decayed = xi_decayed(coherences, f, xis, f_target_idx)    
    plt.ylabel(r'$C_{\xi}$')           
    plt.title(title)
    z_alpha=1
    if 'Noise' in bin_name:
        z_alpha=0.3
    plt.scatter(xis*1000, target_coherences, label=bin_name, alpha=z_alpha, s=s_coherences, color=color)
    plt.scatter(xis[xi_decayed_idx]*1000, target_coherences[xi_decayed_idx], marker='*', s=s_decay, color=color, edgecolors='black', zorder=10)
plt.xlabel(r'$\xi$ [ms]')
plt.legend()

# Num Cycles scale
plt.subplot(2, 1, 2)
title = r"$C_{\xi}$ Decays (# Cycles)"
for f_target_idx, bin_name, color in zip(bin_idxs, bin_names, colors):
    target_coherences = coherences[f_target_idx, :]
    xi_decayed_idx, xi_decayed = xi_decayed(coherences, f, xis, f_target_idx)
    nums_cycles = xis * f[f_target_idx] # Nondimensionalize
    plt.ylabel(r'$C_{\xi}$')           
    plt.title(title)
    z_alpha=1
    if 'Noise' in bin_name:
        z_alpha=0.3
    plt.scatter(nums_cycles, target_coherences, label=bin_name, alpha=z_alpha, s=s_coherences, color=color)
    plt.scatter(nums_cycles[xi_decayed_idx], target_coherences[xi_decayed_idx], marker='*', s=s_decay, color=color, edgecolors='black', zorder=10)
plt.xlabel(r'# of Cycles')
# Prevent us from getting hella noise for the 12kHz one
max_target_f_idx = np.max(bin_idxs[0:-2]) # Get the largest peak frequency
max_num_cycles = xis[-1] * f[max_target_f_idx] # Get corresponding max number of cycles
plt.xlim(0, max_num_cycles) 
plt.legend()

# Book it!
plt.tight_layout()
plt.savefig(f'{fig_folder}C_xi Decays Xi vs # Cycles - {fn_id}.png', dpi=300)
plt.show()
    
